In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
import numpy as np
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import xlsxwriter
from openpyxl import Workbook, load_workbook
import os
import shutil
import logging
import traceback
import re
from azure.storage.blob import BlobServiceClient
# eyJhbGciOiJSUzI1NiIsImtpZCI6IjY3YmFiYWFiYTEwNWFkZDZiM2ZiYjlmZjNmZjVmZTNkY2E0Y2VkYTEiLCJ0eXAiOiJKV1QifQ.eyJuYW1lIjoiV29qdGVrIEJhbG9uIiwicGljdHVyZSI6Imh0dHBzOi8vbGgzLmdvb2dsZXVzZXJjb250ZW50LmNvbS9hL0FBY0hUdGZCODM1WVhSalRJeEl4WmxyTnBaRXpWQk9hZmUyMUFmU1dZZXNnUGc9czk2LWMiLCJpc3MiOiJodHRwczovL3NlY3VyZXRva2VuLmdvb2dsZS5jb20vZXhhMi1mYjE3MCIsImF1ZCI6ImV4YTItZmIxNzAiLCJhdXRoX3RpbWUiOjE2ODY2NTg5MDYsInVzZXJfaWQiOiJEcWRXRDhRdloyUTkzcTR4WFhWWlFWUk8wSEMyIiwic3ViIjoiRHFkV0Q4UXZaMlE5M3E0eFhYVlpRVlJPMEhDMiIsImlhdCI6MTY4NjY1OTA2MSwiZXhwIjoxNjg2NjYyNjYxLCJlbWFpbCI6IndvamJhbDNAZ21haWwuY29tIiwiZW1haWxfdmVyaWZpZWQiOnRydWUsImZpcmViYXNlIjp7ImlkZW50aXRpZXMiOnsiZ29vZ2xlLmNvbSI6WyIxMTUwNTc1NjgzNzI4NjQ1MzA0NTciXSwiZW1haWwiOlsid29qYmFsM0BnbWFpbC5jb20iXX0sInNpZ25faW5fcHJvdmlkZXIiOiJnb29nbGUuY29tIn19.IAOh_U2LXNXGk1jqG3q6m9utI79QVMDtCuUcDBSH5TEKPmMCEdW962qOZN6J8wfMzexHX1cWoqGcXYBmjLcjQKBhhQoAUAdYjxEivrLHe8Hi37bIwXrEX9mvAKD1wE71Sq1sbB3B9xU51lTsH88l7P0pq9LDgbaKkJCljvvzJ186BTbX9Qw0CF4gma1XjJ1W3Nmd0BK2pE9y0b3arF_V8bSME6BeR4Ls1yKLM9da-MCN5y-IkwGVB6j78Qrt-4_emtAhxjkcYlzauOtEM8dZ0NzblgSxY-hdG_sG-Clg0gM6fxXRQSQJYjqHNgwY7sjAP885JUWbtjWjoXKvdJn_iA

In [7]:
# File paths


# date_today = datetime.date.today().strftime("%Y-%m-%d")
# output_viator = r'output/Viator'
# archive_folder = fr'{output_viator}/Archive'
# file_path_done =fr'output/Viator/{date_today}-DONE-Viator.csv'  
# file_path_output = fr"output/Viator - {date_today}.xlsx"
# link_file = fr'resource/Viator_links.csv'
# avg_file = fr'resource/avg-Viator.csv'
# re_run_path = fr'output/Viator/{date_today}-ReRun-Viator.csv'
# folder_path_with_txt_to_count_avg = 'Avg/Viator'
# logs_path = fr'Logs/Viator'

date_today = datetime.date.today().strftime("%Y-%m-%d")
output_musement = r'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Musement/Daily'
archive_folder = fr'{output_musement}/Archive'
file_path_done =fr'{output_musement}/{date_today}-DONE-Musement.csv'  
file_path_output = fr"{output_musement}/Viator - {date_today}.xlsx"
link_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/Musement_links.csv'
# avg_file = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Resource/avgmMusement.csv'
re_run_path = fr'{output_musement}/{date_today}-ReRun-Musement.csv'
logs_path = fr'G:/.shortcut-targets-by-id/1ER8hilqZ2TuX2C34R3SMAtd1Xbk94LE2/MyOTAs/Baza Excel/Logs/Musement'
# FOR ONE TIME USED NOT SYNCHORNIEZD WITH RUNING APPLCIATION
folder_path_with_txt_to_count_avg = 'Avg/Musement'

# Set the path of the local file
local_file_path = f"{output_musement}/Musement - {date_today}.xlsx"

# Set the name of your Azure Storage account and the corresponding access key
storage_account_name = "storagemyotas"
storage_account_key = "vyHHUXSN761ELqivtl/U3F61lUY27jGrLIKOyAplmE0krUzwaJuFVomDXsIc51ZkFWMjtxZ8wJiN+AStbsJHjA=="

# Set the name of the container and the desired blob name
container_name_raw = "raw/daily/musement"
container_name_refined = "refined/daily/musement"

blob_name = fr'Musement - {date_today}.xlsx'



In [8]:
# create logger object
logger_err = logging.getLogger('Error_logger')
logger_err.setLevel(logging.DEBUG)
logger_info = logging.getLogger('Info_logger')
logger_info.setLevel(logging.DEBUG)
logger_done = logging.getLogger('Done_logger')
logger_done.setLevel(logging.DEBUG)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

# create file handler for error logs and set level to debug
fh_error = logging.FileHandler(fr'{logs_path}/error_logs.log')
fh_error.setLevel(logging.DEBUG)

# create file handler for info logs and set level to info
fh_info = logging.FileHandler(fr'{logs_path}/info_logs.log')
fh_info.setLevel(logging.INFO)

# create file handler for info logs and set level to info
fh_done = logging.FileHandler(fr'{logs_path}/done_logs.log')
fh_done.setLevel(logging.INFO)
# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to handlers
ch.setFormatter(formatter)
fh_error.setFormatter(formatter)
fh_info.setFormatter(formatter)
fh_done.setFormatter(formatter)

# add handlers to logger
logger_err.addHandler(ch)
logger_err.addHandler(fh_error)
logger_info.addHandler(ch)
logger_info.addHandler(fh_info)
logger_done.addHandler(ch)
logger_done.addHandler(fh_done)

In [ ]:
# def daily_run_viator(df_links=pd.DataFrame(), re_run=False):

df_links=pd.DataFrame()
re_run=False
global date_today
global output_viator
global file_path_done
global file_path_output
global avg_file
global re_run_path
global folder_path_with_txt_to_count_avg
global archive_folder
#     link_file = fr'resource/Viator_links.csv'
if len(df_links) == 0:
    df_links = pd.read_csv(link_file)
    df_links = df_links.head(20)
######################################################
#         df_links = df_links.head(50)
#         df_links = df_links[(df_links['City'] == 'London') | (df_links['City'] == 'Edinburgh')]
######################################################
# city = df_links.iloc[15]['City']
# category = df_links.iloc[15]['Category']
# t_url
# df_links = df_links[df_links['WhatIsIt'] == 'Category']
# df_links = df_links.head(6)
EUR_City = [
    "Amsterdam", "Athens", "Barcelona", "Berlin", "Dublin", "Dubrovnik", "Florence", "Istanbul",
    "Krakow", "Lisbon", "Madrid", "Milan", "Naples", "Paris", "Porto", "Rome", "Palermo", "Venice",
    "Taormina", "Capri", "Sorrento", "Mount-Etna", "Mount-Vesuvius", "Herculaneum", "Amalfi-Coast",
    "Pompeii"
]

USD_City = [
    "Las-Vegas", "New-York-City", "Cancun", "Dubai"
]

GBP_City = [
    "Edinburgh", "London"
]

#     date_today = datetime.date.today().strftime("%Y-%m-%d")
#     file_path_done =fr'output/Viator/{date_today}-DONE-Viator.csv'  
#     file_path_output = f"output/Viator - {date_today}.xlsx"
if os.path.exists(file_path_output) and re_run == False:
    print(f'Today ({date_today}) Viator done')
#     return 'Done'



if os.path.exists(file_path_done) and re_run == False:
    done_msg = pd.read_csv(file_path_done)
    done_msg = done_msg.transpose()
    done_msg = done_msg.set_axis(done_msg.iloc[0], axis=1)
    done_msg = done_msg.iloc[1:]
    done_index = int(done_msg.index.values[0])
    df_links = df_links.iloc[(done_index+1):]
#         display(df_links)
elif re_run == True:
    print(f'Lenght of links: {len(df_links)}')
else:
    print("Nothing done yet")
options = webdriver.ChromeOptions()
#     options.add_argument('--blink-settings=imagesEnabled=false')
#     options.add_argument("--disable-gpu")
#     options.add_argument("--no-sandbox")
#     options.add_argument("--log-level=3")
options.add_argument("--disable-infobars")
options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches",["enable-automation"])
#     options.add_argument("--user-data-dir=C:/Users/Wojciech/AppData/Local/Google/Chrome/User Data/Default")
#     options.add_argument('start-maximized')
#     options.add_argument('disable-infobars')
#     options.add_argument("--disable-logging")
#     options.add_argument("--disable-login-animations")
#     options.add_argument("--disable-notifications")
#     options.add_argument("--disable-default-apps")
#     options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.maximize_window()
# Define the URL of the website we want to scrape
start_time = time.time()
total_pages = 0
max_to_do = 0
if len(df_links) == 0:
#     return 'Done'
    pass
for index, row in df_links.iterrows():
#     CHECK IF FILE PATH EXISIT IF SO CHECK THE DATA INSIDE
    print(row["URL"])

    data = []
    position = 0
    url_time = time.time()
    url = f'{row["URL"]}'
    driver.get(url)
#         print(url)
    currency = driver.find_element(By.CSS_SELECTOR, ("[data-type=currency]")).text.strip()
#         currency = soup.select_one("[data-type=currency]").text.strip()

    if row['City'] in EUR_City:
        if '€' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='EUR']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)

    elif row['City'] in USD_City:
        if '$' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='USD']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)
    elif row['City'] in GBP_City:
        if '£' in currency:
            pass
        else:
            # Assuming you have initialized the webdriver and assigned it to the 'driver' variable
            button = driver.find_element(By.XPATH, "//input[@type='radio' and @value='GBP']")
            # Scroll to the element not requried
            # driver.execute_script("arguments[0].scrollIntoView();", element)
            # Click the element
            driver.execute_script("arguments[0].click();", button)

    else:
        print('Missing from the list:', city)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    try:
        product_list_count = int(soup.select_one("[data-test-id=activity-count-text]").text.strip().split()[0])
        max_pages = int(round(int(product_list_count)/12,0) + 1)
    except:
        print('Max page cannot be found')
    driver.get(url)
    time.sleep(1)
    products = ''
    print('product_list_count:', product_list_count)
    while len(products) < product_list_count:
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        products = soup.select("[data-test=ActivityCard]")
        print(len(products))
        try:
            more_activities = driver.find_element(By.CSS_SELECTOR, 'button[data-test="test-btn"]')
            driver.execute_script("arguments[0].click();", more_activities)
            time.sleep(1)
        except:
            print('E')
            time.sleep(2)
            
    print('Products check')
    for product in products:
        try:
            title = product.find('a').text.strip() # Title
        except:
            print(f'For url {url} collected {position}')
            break
        product_url = product.find('a')['href'] #URL
        price = product.select_one("[data-test=realPrice]").text # Price
        position = position + 1
        siteuse = 'Musement'
        city = row['City']
        category = row['Category']
        try:
            stars = product.select_one("[data-test=reviewTest]").find('strong').text.strip() # Ratings
        except:
            stars = 'N/A'
        try:
            amount_reviews = product.select_one("p[class*=reviewsNumber]").text.replace('(','').replace(')','') # Amount of reviews
        except:
            amount_reviews = 'N/A'
        
        data.append([title,product_url, price, stars, amount_reviews, date_today, position, category, siteuse, city ])
        
    url_done = time.time()
    message = f'Time for {city}-{category}: {round((url_done - url_time)/60, 3)}min | Pages: {max_pages} | AVG {round((url_done - url_time)/max_pages, 2)}s per page Currency: 1-{first_style_curr}, 2-{second_style_curr}, 3-{thirtd_style_curr}'
    print(message)
    logger_info.info(message)
    df = pd.DataFrame(data, columns=['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto'])
    
    file_path = fr'{output_musement}/{date_today}-{city}-Musement.csv' 
    df.to_csv(file_path, header=not os.path.exists(file_path), index=False, mode='a')
    row.to_csv(file_path_done, header=True, index=True)    

Nothing done yet
https://www.musement.com/uk/amsterdam/?sort_by=-relevance-city&limit=12
product_list_count: 363
12
24
36
48
48
48
0
12
24
36
48
48
48
48
48
12
24
36
0
12
24
24
12
24
24
24
12
24
36
48
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E
51
E

In [60]:
df = pd.DataFrame(data, columns=['Tytul', 'Tytul URL', 'Cena', 'Opinia', 'IloscOpini', 'Data zestawienia', 'Pozycja', 'Kategoria', 'SiteUse', 'Miasto'])
df

,Tytul,Tytul URL,Cena,Opinia,IloscOpini,Data zestawienia,Pozycja,Kategoria,SiteUse,Miasto
0,Van Gogh Museum ticket,/uk/amsterdam/van-gogh-museum-ticket-651/,€22.00,4.4,6467,2023-06-28,1,Global,Musement,Amsterdam
1,ARTIS Amsterdam Royal Zoo entrance ticket,/uk/amsterdam/artis-amsterdam-royal-zoo-entran...,€25.00,4.7,77,2023-06-28,2,Global,Musement,Amsterdam
2,Rijksmuseum entrance ticket,/uk/amsterdam/rijksmuseum-entrance-ticket-91921/,€23.50,4.7,303,2023-06-28,3,Global,Musement,Amsterdam
3,Heineken® Experience tickets,/uk/amsterdam/heineken-r-experience-tickets-2224/,€21.00,4.5,350,2023-06-28,4,Global,Musement,Amsterdam
4,Ticket to AMAZE immersive audiovisual experience,/uk/amsterdam/ticket-to-amaze-immersive-audiov...,€26.95,4.6,5,2023-06-28,5,Global,Musement,Amsterdam
...,...,...,...,...,...,...,...,...,...,...
355,Europe audio guide with TravelMate app,/uk/rome/europe-audio-guide-with-travelmate-ap...,€36.90,N/A,N/A,2023-06-28,356,Global,Musement,Amsterdam
356,The Netherlands private day trip to the Dutch ...,/uk/amsterdam/the-netherlands-private-day-trip...,€590.00,N/A,N/A,2023-06-28,357,Global,Musement,Amsterdam
357,The Story of Edam Cheese museum entrance ticket,/uk/volendam/the-story-of-edam-cheese-museum-e...,€8.00,5,1,2023-06-28,358,Global,Musement,Amsterdam
358,Holland four cities private day-trip,/uk/amsterdam/holland-four-cities-private-day-...,€110.00,N/A,N/A,2023-06-28,359,Global,Musement,Amsterdam


In [67]:
product = products[208]
product_url = product.find('a')['href'] #URL
stars = product.select_one("[data-test=reviewTest]").find('strong').text.strip() # Ratings
amount_reviews = product.select_one("p[class*=reviewsNumber]").text.replace('(','').replace(')','') # Amount of reviews
price = product.select_one("[data-test=realPrice]").text # Price
title = product.find('a').text.strip() # Title

# data.append([title,product_url, price, stars, amount_reviews, discount, text, date_today, position, category, siteuse, city ])

AttributeError: 'NoneType' object has no attribute 'find'

In [64]:
title

'2-hour small-group Rijksmuseum guided tour'